# Assignment 7
* analizzare i servizi forniti da **CoinGecko**
* scaricare il client fonito da **CoinGecko** e analizzarne la funzionalità
* scrivere uno script **Python** che implementi le seguenti funzionalità
    * ricercare gli exchanger tracciati da **CoinGecko** e scriverli in un file **JSON**
    * memorizzare, per ogni exchanger, il volume di mercato relativo a **Bitcoin** nella data corrente
* facoltativo: scaricare un insieme di dati a scelta da **CoinGecko** e effettuare un'analisi a scelta su tale dati

In [48]:
import pandas as pd
from pycoingecko import CoinGeckoAPI
import json
import time
import csv
import os
import datetime
import warnings
warnings.filterwarnings('ignore')


In [47]:
cg = CoinGeckoAPI()

exchanges = cg.get_exchanges_list()
json_file = {}

# print(exchanges)

for exch in exchanges:
    exchange_id = exch["id"]
    market_volume = exch['trade_volume_24h_btc']
    json_file[exchange_id] = {
        'name' : exch['name'] ,
        'market_volume_btc' : market_volume ,
        'date' : str(date.today())
    }

with open('exchanges_BTC_market_volume.json', 'w') as file:
    json.dump(json_file , file, indent=4)


In [49]:
def get_historical_prices(coin_id, start_date, end_date):
    # Fetch historical price data from CoinGecko API
    historical_data = cg.get_coin_market_chart_range_by_id(
        id=coin_id,
        vs_currency='usd',
        from_timestamp=start_date,
        to_timestamp=end_date
    )
    # Extract prices from the returned data
    prices = historical_data['prices']
    # Convert the list of tuples into a DataFrame
    df = pd.DataFrame(prices, columns=['timestamp', 'price'])
    # Convert Unix timestamp to datetime
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    # Set timestamp as index
    df.set_index('timestamp', inplace=True)
    return df


start_date = pd.Timestamp('2018-01-01')
end_date = pd.Timestamp('2019-01-01')

# Fetch historical prices for Bitcoin
bitcoin_prices = get_historical_prices('bitcoin', start_date.timestamp(), end_date.timestamp())
# Fetch historical prices for Ethereum
ethereum_prices = get_historical_prices('ethereum', start_date.timestamp(), end_date.timestamp())

# Merge
df = pd.merge(bitcoin_prices, ethereum_prices, on='timestamp', suffixes=('_bitcoin', '_ethereum'))

df["cambio_percentuale_btc"] = df["price_bitcoin"].pct_change() * 100
df["cambio_percentuale_eth"] = df["price_ethereum"].pct_change() * 100

btc_start_value = df["price_bitcoin"][0]
eth_start_value = df["price_ethereum"][0]

media_btc = df["cambio_percentuale_btc"].mean()
media_eth = df["cambio_percentuale_eth"].mean()

btc_pos_changes = df[ df["cambio_percentuale_btc"] > 0].count()["cambio_percentuale_btc"]
eth_pos_changes = df[ df["cambio_percentuale_eth"] > 0].count()["cambio_percentuale_eth"]

print(f"Analisi andamento di mercato nel 2018 di btc e eth")
print(f"Prezzo di partenza in USD: BTC:{btc_start_value}$ , ETH:{eth_start_value}$")
print(f"Medio andamento% da giorno a giorno: BTC: {media_btc} ETH: {media_eth}")
print(f"Numero di giorni di aumento rispetto al precedente: BTC: {btc_pos_changes}, ETH:{eth_pos_changes}")
print(f"Numero di giorni di perdita rispetto al precedente: BTC: {365 - btc_pos_changes}, ETH:{365 - eth_pos_changes}")

Analisi andamento di mercato nel 2018 di btc e eth
Prezzo di partenza in USD: BTC:14093.6068305274$ , ETH:776.080330355968$
Medio andamento% da giorno a giorno: BTC: -0.27433901511417447 ETH: -0.3278919531384932
Numero di giorni di aumento rispetto al precedente: BTC: 185, ETH:170
Numero di giorni di perdita rispetto al precedente: BTC: 180, ETH:195
